<a href="https://colab.research.google.com/github/umutcankarakas/lstmmusicgenerator/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/umutcankarakas/theorygenerator.git

Cloning into 'theorygenerator'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 51 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (51/51), done.


In [ ]:
!pip install keras-self-attention

  Created wheel for keras-self-attention: filename=keras_self_attention-0.46.0-cp36-none-any.whl size=17278 sha256=c34af8d65a0f0ea86204d71c9489b81e65235bcab791cfaecee19623d6327367
  Stored in directory: /root/.cache/pip/wheels/d2/2e/80/fec4c05eb23c8e13b790e26d207d6e0ffe8013fad8c6bdd4d2
Successfully built keras-self-attention


In [ ]:
import os
import numpy as np
from pathlib import *
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.layers import Dense, Dropout, LSTM, Activation, Bidirectional, Flatten
#from keras import utils
from keras.callbacks import ModelCheckpoint
from keras_self_attention import SeqSelfAttention
from matplotlib import pyplot as plt
import string

Using TensorFlow backend.


In [ ]:
os.chdir('theorygenerator')

In [ ]:
train_path = '../drive/My Drive/full_set_beethoven_mozart/'

In [ ]:
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = 128 + 128 + 2

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)
    history = train(model, network_input, network_output)
    return history

In [ ]:
def convertMidi2Arr(midi_element):
  newlist = []
  max_duration = 8*4*4
  if isinstance(midi_element, note.Note):
    newlist.append(midi_element.pitch.midi)
    if int(round(midi_element.duration.quarterLength*4)) > max_duration:
      newlist.append(128 + max_duration)
    else:
      newlist.append(128 + int(round(midi_element.duration.quarterLength*4)))
  elif isinstance(midi_element, chord.Chord):
    for n in midi_element:
      newlist.append(n.pitch.midi) 
      if int(round(midi_element.duration.quarterLength*4)) > max_duration:
        newlist.append(128 + max_duration)
      else:
        newlist.append(128 + int(round(midi_element.duration.quarterLength*4)))
  elif isinstance(midi_element, note.Rest):
    newlist.append(128)
    if int(round(midi_element.duration.quarterLength*4)) > max_duration:
      newlist.append(128 + max_duration)
    else:
      newlist.append(128 + int(round(midi_element.duration.quarterLength*4)))
  return newlist

In [ ]:
def get_notes():
  mylist = []
  count = 0
  last_offset = 0.0
  offset_diff = 0.0
  for file in glob.glob(train_path + '**/*.mid', recursive=True):
    try:
      midi = converter.parse(file)
    except:
      continue
    #print("Parsing %s" % file)
    #print(count)
    notes_to_parse = None
    max_duration = 8*4*4

    try: # file has instrument parts
      s2 = instrument.partitionByInstrument(midi_piece)
      notes_to_parse = s2.parts[0].recurse() 
    except: # file has notes in a flat structure
      notes_to_parse = midi.flat.notesAndRests

    same_offset_elements = []
    #min_duration = max_duration
    for element in notes_to_parse:
      if isinstance(element, note.Note) or isinstance(element, chord.Chord) or isinstance(element, note.Rest):
        if( not same_offset_elements or same_offset_elements[0].offset == element.offset):
          same_offset_elements.append(element)
        else:
          add_arr = []
          #flags = [0,0,0]
          for toadd_midi in same_offset_elements:
            toadd = convertMidi2Arr(toadd_midi)
            if toadd[0]==128:
              continue
            add_arr.extend(toadd)
          if(element.offset > last_offset):
            offset_diff = same_offset_elements[0].offset - last_offset
            last_offset = same_offset_elements[0].offset
          if mylist and mylist[-1] != '_':
            mylist.append(257)
            mylist.append(128+int(round(offset_diff*4)))
            mylist.extend(add_arr)
          else:
            mylist.extend(add_arr)

          offset_diff = 0.0
          same_offset_elements = []
          same_offset_elements.append(element)

    if same_offset_elements:
      add_arr = []
      for toadd_midi in same_offset_elements:
        toadd = convertMidi2Arr(toadd_midi)
        if toadd[0]==128:
          continue
        add_arr.extend(toadd)
      if(element.offset > last_offset):
        offset_diff = same_offset_elements[0].offset - last_offset
        last_offset = same_offset_elements[0].offset
      if mylist and mylist[-1] != '_':
        mylist.append(257)
        mylist.append(128+int(round(offset_diff*4)))
        mylist.extend(add_arr)
      else:
        mylist.extend(add_arr)

    if count != 0: count = count + 1
    else: break
    mylist.append('_')

  with open('../drive/My Drive/data/notes_attention_mozbet', 'wb') as filepath:
    pickle.dump(mylist, filepath)
  return mylist

In [ ]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 400

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        if '_' in sequence_in:
          i = sequence_in.index('_') + 1
          continue
        elif sequence_out == '_':
          i = i + sequence_length +1
          continue
        
        network_input.append([element for element in sequence_in])
        network_output.append(sequence_out)

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [ ]:
def create_network_1(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(512))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    if(os.path.exists('../drive/My Drive/bach_poins/weghts-improvement-292-0.2710-bigger.hdf5')): model.load_weights('../drive/My Drive/bach_points/weights-improvement-292-0.2710-bigger.hdf5')
    return model

In [ ]:
def create_network_2(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(Bidirectional(LSTM(512,
        input_shape=(network_input.shape[1], network_input.shape[2]), #n_time_steps, n_features?
        return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(LSTM(512,return_sequences=True))
    model.add(Flatten()) #Supposedly needed to fix stuff before dense layer
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    if(os.path.exists('../drive/My Drive/bach_points/weghts-improvement-292-0.2710-bigger.hdf5')): model.load_weights('../drive/My Drive/bach_points/weights-improvement-292-0.2710-bigger.hdf5')
    return model

In [ ]:
def create_network_3(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(512, return_sequences=True))
    model.add(LSTM(512))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    if(os.path.exists('../drive/My Drive/checkpoints_6lstm_se400only_t/weights-024-0.8723.hdf5')): model.load_weights('../drive/My Drive/checkpoints_6lstm_seq400_only_t/weights-024-0.8723.hdf5')
    return model

In [ ]:
def create_network_4(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.2,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.2,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.2))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    if(os.path.exists('../drive/My Drive/bach_poins/weghts-improvement-292-0.2710-bigger.hdf5')): model.load_weights('../drive/My Drive/bach_points/weights-improvement-292-0.2710-bigger.hdf5')
    return model

In [ ]:
def create_network_5(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(Bidirectional(LSTM(512,
        input_shape=(network_input.shape[1], network_input.shape[2]), #n_time_steps, n_features?
        return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(LSTM(512,return_sequences=True))
    model.add(Flatten()) #Supposedly needed to fix stuff before dense layer
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    if(os.path.exists('../drive/My Drive/bach_ponts/weghts-improvement-292-0.2710-bigger.hdf5')): model.load_weights('../drive/My Drive/bach_points/weights-improvement-292-0.2710-bigger.hdf5')
    return model

In [ ]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(Bidirectional(LSTM(512,
        input_shape=(network_input.shape[1], network_input.shape[2]), #n_time_steps, n_features?
        return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    #model.add(Dropout(0.3))
    
    model.add(LSTM(512,return_sequences=True))
    #model.add(Dropout(0.3))
    
    model.add(Flatten()) #Supposedly needed to fix stuff before dense layer
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    if(os.path.exists('../drive/My Drive/checkpoints_attentio/weights-00-0.0274.hdf5')): model.load_weights('../drive/My Drive/checkpoints_attention/weights-200-0.0274.hdf5')
    return model

In [ ]:
def create_network_7(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(Bidirectional(LSTM(512,
        input_shape=(network_input.shape[1], network_input.shape[2]), #n_time_steps, n_features?
        return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(Dropout(0.3))
    
    model.add(LSTM(512,return_sequences=True))
    model.add(Dropout(0.3))
    
    model.add(Flatten()) #Supposedly needed to fix stuff before dense layer
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    if(os.path.exists('../drive/My Drive/checkpoints_attentio/weights-00-0.0274.hdf5')): model.load_weights('../drive/My Drive/checkpoints_attention/weights-200-0.0274.hdf5')
    return model

In [ ]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = os.path.abspath("../drive/My Drive/checkpoints_attention_mozbet/weights-{epoch:03d}-{loss:.4f}.hdf5")
    
    checkpoint = ModelCheckpoint(
        filepath,
        period=10, #Every epoch
        monitor='loss',
        verbose=1,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    history = model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list, validation_split=0.2, shuffle=True)
    return history

In [ ]:
history = None
if __name__ == '__main__':
    history = train_network()

Train on 4534 samples, validate on 1134 samples
Epoch 1/200
4534/4534 [==============================] - 168s 37ms/step - loss: 3.9690 - accuracy: 0.2333 - val_loss: 3.1537 - val_accuracy: 0.3095
Epoch 2/200
4534/4534 [==============================] - 166s 37ms/step - loss: 3.2347 - accuracy: 0.2574 - val_loss: 3.0650 - val_accuracy: 0.3095
Epoch 3/200
4534/4534 [==============================] - 165s 36ms/step - loss: 3.0614 - accuracy: 0.2750 - val_loss: 3.0349 - val_accuracy: 0.1720
Epoch 4/200
4534/4534 [==============================] - 164s 36ms/step - loss: 3.0411 - accuracy: 0.2620 - val_loss: 2.9948 - val_accuracy: 0.3095
Epoch 5/200
4534/4534 [==============================] - 164s 36ms/step - loss: 2.9887 - accuracy: 0.2922 - val_loss: 3.1070 - val_accuracy: 0.1720
Epoch 6/200
4534/4534 [==============================] - 164s 36ms/step - loss: 2.9803 - accuracy: 0.2759 - val_loss: 3.0453 - val_accuracy: 0.3095
Epoch 7/200
4534/4534 [==============================] - 163s 36

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
#len(glob.glob(train_path + '**/*nokey.mid', recursive=True))